In [18]:
import pickle
import os

In [34]:
print ('loading weights state')
f = open('weights.save', 'rb')
loaded_objects = []
for i in range(5):
    loaded_objects.append(pickle.load(f,encoding='latin1'))
f.close()
loaded_objects[4][0]

loading weights state


array([[  1.38502998e-07,  -3.98696699e-07,   2.13523862e-07, ...,
          4.68765258e-08,   1.93015843e-08,  -1.63970206e-08],
       [ -2.28999648e-02,   6.36817189e-03,   3.53702791e-02, ...,
         -1.43225669e-04,  -8.77054874e-04,   1.06765972e-02],
       [ -5.93338199e-02,   8.20592791e-02,  -2.15487123e-01, ...,
          1.69472277e-01,   4.33959393e-03,  -1.81922317e-01],
       ..., 
       [ -2.15400998e-02,  -2.40251366e-02,   8.10766146e-02, ...,
          1.62847782e-03,   9.53744501e-02,   2.06153482e-01],
       [ -2.44145184e-07,   5.20831556e-08,   3.37133997e-07, ...,
         -2.09719161e-07,  -2.32480915e-07,  -2.68581147e-07],
       [  2.93973955e-07,  -1.08248104e-08,   5.01130764e-08, ...,
         -1.12287502e-07,  -9.10215547e-08,  -2.67783150e-07]], dtype=float32)

In [20]:
for loadedweight in loaded_objects :
    print(loadedweight[0].shape,loadedweight[1].shape)


(40, 20, 5, 5) (40,)
(60, 40, 3, 3) (60,)
(90, 60, 3, 3) (90,)
(1440, 500) (500,)
(500, 8) (8,)


In [21]:
import numpy as np
nppara = np.array(loaded_objects)

In [22]:
nppara.shape

(5, 2)

In [23]:
nppara[0,0].shape

(40, 20, 5, 5)

In [1]:
import cv2
import numpy as np
import tensorflow as tf
#import theano.tensor as T
#from theano.tensor.signal import downsample
#from theano.tensor.nnet import conv

/Users/wangzhouhao/anaconda2/envs/python36nlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
inputs = np.arange(50000)

In [35]:
x = tf.placeholder(dtype='int32')
y = tf.reshape(x,(1,20,50,50))
z = tf.transpose(y, perm=[0,2,3,1])
shape = tf.shape(z)

with tf.Session() as sess :
    outshape,output = sess.run([shape,z],feed_dict={x:inputs})

In [38]:
print(output[0][0])

[[    0  2500  5000  7500 10000 12500 15000 17500 20000 22500 25000 27500
  30000 32500 35000 37500 40000 42500 45000 47500]
 [    1  2501  5001  7501 10001 12501 15001 17501 20001 22501 25001 27501
  30001 32501 35001 37501 40001 42501 45001 47501]
 [    2  2502  5002  7502 10002 12502 15002 17502 20002 22502 25002 27502
  30002 32502 35002 37502 40002 42502 45002 47502]
 [    3  2503  5003  7503 10003 12503 15003 17503 20003 22503 25003 27503
  30003 32503 35003 37503 40003 42503 45003 47503]
 [    4  2504  5004  7504 10004 12504 15004 17504 20004 22504 25004 27504
  30004 32504 35004 37504 40004 42504 45004 47504]
 [    5  2505  5005  7505 10005 12505 15005 17505 20005 22505 25005 27505
  30005 32505 35005 37505 40005 42505 45005 47505]
 [    6  2506  5006  7506 10006 12506 15006 17506 20006 22506 25006 27506
  30006 32506 35006 37506 40006 42506 45006 47506]
 [    7  2507  5007  7507 10007 12507 15007 17507 20007 22507 25007 27507
  30007 32507 35007 37507 40007 42507 45007 47507]


In [31]:
print(outshape)

[ 1 20 50 50]


In [32]:
W = tf.Variable(initial_value=loaded_objects[4][0], name='W')
shape = tf.shape(W)

with tf.Session() as sess :
    init = tf.global_variables_initializer()
    sess.run(init)
    outshape,W = sess.run([shape,W],feed_dict={})

In [26]:
outshape

array([500,   8], dtype=int32)

In [27]:
W

array([[  1.38502998e-07,  -3.98696699e-07,   2.13523862e-07, ...,
          4.68765258e-08,   1.93015843e-08,  -1.63970206e-08],
       [ -2.28999648e-02,   6.36817189e-03,   3.53702791e-02, ...,
         -1.43225669e-04,  -8.77054874e-04,   1.06765972e-02],
       [ -5.93338199e-02,   8.20592791e-02,  -2.15487123e-01, ...,
          1.69472277e-01,   4.33959393e-03,  -1.81922317e-01],
       ..., 
       [ -2.15400998e-02,  -2.40251366e-02,   8.10766146e-02, ...,
          1.62847782e-03,   9.53744501e-02,   2.06153482e-01],
       [ -2.44145184e-07,   5.20831556e-08,   3.37133997e-07, ...,
         -2.09719161e-07,  -2.32480915e-07,  -2.68581147e-07],
       [  2.93973955e-07,  -1.08248104e-08,   5.01130764e-08, ...,
         -1.12287502e-07,  -9.10215547e-08,  -2.67783150e-07]], dtype=float32)